In [9]:
%pip install -U -q pandas dspy datasets

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import dspy  
  
# Initialize ColBERTv2 with a URL to a ColBERT server  
colbert = dspy.ColBERTv2(url='http://20.102.90.50:2017/wiki17_abstracts')  
  
# Configure DSPy to use this retriever as the default  
dspy.configure(rm=colbert)  
  
# Use the retriever directly  
results = colbert("What is capital of France", k=3)  
  
# Or use it with a DSPy module  
# retrieve = dspy.Retrieve(k=3)  
# results = retrieve("What is retrieval-augmented generation?")

In [8]:
results

[{'text': 'Capital (French magazine) | Capital is a monthly French economics and business magazine published in Paris, France.',
  'pid': 3723041,
  'rank': 1,
  'score': 25.811302185058594,
  'prob': 0.3812346890513862,
  'long_text': 'Capital (French magazine) | Capital is a monthly French economics and business magazine published in Paris, France.'},
 {'text': 'Paris (disambiguation) | Paris is the largest city and capital of France.',
  'pid': 1578992,
  'rank': 2,
  'score': 25.610862731933594,
  'prob': 0.3119914283281822,
  'long_text': 'Paris (disambiguation) | Paris is the largest city and capital of France.'},
 {'text': 'Paris | Paris (] ) is the capital and most populous city of France, with an administrative-limits area of 105 km2 and a 2015 population of 2,229,621. The city is a commune and department, and the capital-heart of the 12,012 km2 Île-de-France "region" (colloquially known as the \'Paris Region\'), whose 12,142,802 2016 population represents roughly 18 percent o

In [22]:
import dspy  
from dspy.datasets import HotPotQA  
  
# Configure LM and RM  
lm = dspy.LM('openai/gpt-4o-mini')  
colbert = dspy.ColBERTv2(url='http://20.102.90.50:2017/wiki17_abstracts')  
dspy.configure(lm=lm, rm=colbert)  
  
# Define a simple RAG module  
class RAG(dspy.Module):  
    def __init__(self, num_docs=3):  
        super().__init__()  
        self.retrieve = dspy.Retrieve(k=num_docs)  
        self.generate = dspy.ChainOfThought("context, question -> answer")  
          
    def forward(self, question):  
        passages = self.retrieve(question).passages  
        context = "\n\n".join(passages)  
        print(f"Context: {context}")
        return self.generate(context=context, question=question)  
  
# Use the RAG module  
rag = RAG()  
# result = rag(question="What is capital of France?")  
# result = rag(question="Where is yoga capital of the world?")  
result = rag(question="What is closest park near Google Headquarters?")  
print(result.answer)

Context: Googleplex | The Googleplex is the corporate headquarters complex of Google, Inc. and its parent company Alphabet Inc., located at 1600 Amphitheatre Parkway in Mountain View, Santa Clara County, California, United States, near San Jose.

Android lawn statues | The Android lawn statues are a series of large foam statues near the Googleplex (Google's headquarters) in Mountain View, California, currently located at 1981 Landings Drive. They are based on the code names for versions of Google's Android mobile operating system, which are named after desserts and sweet treats. These statues were originally located in front of Building 44, where the Android development team had its offices. However, they were moved to their present location a few blocks away (at some point before October 19, 2014), they are now next to the Google Visitor Center Beta which is open to Google employees and their guests. The Android statues remain a tourist attraction where visitors are encouraged to come

In [23]:
result

Prediction(
    reasoning='The context provided does not specify the exact parks near the Googleplex. However, it is known that the Googleplex is located in Mountain View, California, which has several parks in the vicinity. One of the closest parks to the Googleplex is Shoreline Park, which is a large park offering various recreational activities and is located just a short distance away from the headquarters.',
    answer='The closest park near Google Headquarters (the Googleplex) is Shoreline Park.'
)